# RAG 

#### Libraries

In [566]:
# processing:
import os
import re
import pandas as pd
from dotenv import load_dotenv

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain

#### .env

In [4]:
load_dotenv()

True

In [568]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
LLM_SCOPE = os.getenv('SCOPE')
LLM_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [569]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up Database | Neo4j

In [ ]:
embeddings = GigaChatEmbeddings(credentials = LLM_AUTH, verify_ssl_certs = False)

In [ ]:
neo4j_vector = Neo4jVector.from_existing_index(embeddings,
                                               url=NEO4J_URI,
                                               username=NEO4J_USERNAME,
                                               password=NEO4J_PASSWORD,
                                               index_name="Document",
                                               node_label="Document" ,
                                               keyword_index_name = "keywords",
                                               search_type = "hybrid"
                                               )

In [ ]:
retriever = neo4j_vector.as_retriever(search_kwargs={"k": 5, 'score_treshold': 0.9, 'lambda_mult': 0.25})

## Setting Up LLM | GigaChat


In [570]:
llm = GigaChat(model = "GigaChat-Plus",
               temperature = 0.3,
               top_p = 0.2,
               n = 1,
               repetition_penalty = 1,
               credentials = LLM_AUTH, 
               verify_ssl_certs = False
               )

In [571]:
parser = StrOutputParser()

## Prompt Engineering

#### Prompt Template

In [594]:
template = """
            Задача: Анализировать запрос и предоставлять детализированные ответы, используя указанные документы.

            Инструкции:

            1. Цитируйте конкретные пункты и статьи документов при ответе на вопросы, связанные с правилами или нормативными актами.
            3. Объясните применение документов к запросу для точности ответа.
            4. Не повторяй информацию и грамонто формулируй ответы.

            Запрос: \"{question}\"
            Документы: \"{context}\"
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше инструкции.
            Ответ должен сожерджать не более 200 слов и включать финальный вывод.
            
            ВАЖНО: При отсутствии данных для какого-либо из пунктов, просто пропусти упоминание об этом, чтобы поддерживать чистоту и релевантность выходных данных. 
            """

prompt = ChatPromptTemplate.from_template(template)

#### Question Extraction

In [589]:
def extract_question(model, parser, text):
    
    chain = model | parser

    template = f"""
            Задача: Проанализируй представленный запрос для извлечения инфомрации.
            
            Вот текст запроса: \"{text}\"
            
            Проанализируй контекст и явно сформулируй полные вопросы содерджащие ключевые слова без отсылок. 
            Если в тексте явно поставлены вопросы, извлеки их. 
            Если текст не содержит явных вопросов, попытайся сформулировать вопросы, на основании предоставленной информации. 

            ВАЖНО: Результаты должны быть представлены как список уникальных данных, разделенных запятыми, без использования заголовков, разделов или нумерации. 
                   Сосредоточь внимание на ясности и конкретике представленной информации.
            
            Без комментириев.
            """

    model_response = chain.invoke(template)
    
    return model_response

## LLM Retrival Chain

#### Chain

In [577]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = retriever,
                                                    return_source_documents = True,
                                                    reduce_k_below_max_tokens = True,
                                                    max_tokens_limit = 32000,
                                                    chain_type_kwargs = {"verbose": False,
                                                                         "prompt": prompt,
                                                                         "document_variable_name": "context"
                                                                        }
                                                    )

##### Letters + Answers + Questions + Response

In [593]:
letter = 5 # from the reference.xlsx num + 2

inquiry = str(reference['letter'][letter-1].replace('\n', ' '))
refer = str(reference['answer '][letter-1]).replace('\n', ' ')

question = extract_question(llm, parser, inquiry).replace("/", " ")

print(f'Letter: \n\n{inquiry}\n')
print(f'Answer: \n\n{refer}\n')
print(f'Questions: \n\n{question}')

chain.invoke({"question": question}, return_only_outputs = False)

Letter: 

Добрый день! Подскажите, пожалуйста, по дате заключения договора ВЗЛ. Например: Дата утверждения строки ГПЗ 20.02.2023 Планируемая дата подведения итогов в ГПЗ указана 01.03.2023 Дата заключения договора должна быть строго 01.03.2023 или может быть в промежутке с 20.02.2023 по 31.03.2023?

Answer: 

Датой подведения итогов закупки для способа закупки Прочие (ВЗЛ) является дата заключения договора, она же является датой инициации процедуры. В соответствии с п 5.3.6. и 5.3.7. СК-03.02.01 Формирование Годового плана закупок – заказчик контролирует факт инициации запланированных в утвержденном ГПЗ закупок, не инициированные в рамках месяца или ранее закупки подлежат переносу на более поздний срок в рамках года или удалению, не позднее 2 р.д. с начала месяца, следующего за месяцем планируемой инициации. В вашем примере закупка может быть инициирована не позднее 31.03.2023, факт инициации подтверждается внесением информации о заключенном договоре в АС Отчетность.

Questions: 

1. К

{'question': '1. Какая дата заключения договора ВЗЛ?\n2. Может ли дата заключения договора быть в промежутке с 20.02.2023 по 31.03.2023?',
 'answer': '1. Дата заключения договора ВЗЛ не указана в предоставленных документах.\n2. Да, дата заключения договора может быть в промежутке с 20.02.2023 по 31.03.2023, если это предусмотрено законодательством и таким договором.',
 'sources': '',
 'source_documents': [Document(page_content='22.1. Стороны договора вправе вносить изменения и (или) дополнения в договор, заключенный по результатам закупки (далее - изменения и дополнения), а также расторгать заключенный по результатам закупки договор в порядке и по основаниям, предусмотренным законодательством и таким договором.', metadata={'source': 'ПОЛОЖЕНИЕ/22. ОСОБЕННОСТИ ИЗМЕНЕНИЯ, ДОПОЛНЕНИЯ И РАСТОРЖЕНИЯ  ДОГОВОРА', 'bullet': '22.1'}),
  Document(page_content='5.6.2. Если решение о закупке у ЕдП утверждено, Заказчик может принять решение об отказе от заключения договорного документа по итогам пр

## Test

In [597]:
responses = pd.DataFrame(columns = ['letter', 'extracted questions', 'answer', 'rag answers', 'sources'])

for letter in range(1, 21):
    inquiry = str(reference['letter'][letter-1].replace('\n', ' '))
    refer = str(reference['answer '][letter-1]).replace('\n', ' ')
    question = extract_question(llm, parser, inquiry).replace("/", " ")
    response = chain.invoke({"question": question}, return_only_outputs = False)

    answer = response['answer']
    sources = [doc.metadata for doc in response['source_documents']]

    responses = pd.concat([responses, pd.DataFrame({'letter': [inquiry], 
                                                    'extracted questions': [question], 
                                                    'answer': [refer], 
                                                    'rag answers': [answer], 
                                                    'sources': [sources]})
                           ], ignore_index = True)

Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


In [582]:
responses.head(3)

,letter,extracted questions,answer,rag answers,sources
0,"Уважаемые коллеги, Добрый день! У меня возник...",1. Можно ли при расчете НМЦ указывать ориентир...,"Согласно пункту 5.11 ПоЗ, при закупках товаров...",На основании предоставленных документов можно ...,"[{'source': 'ГПЗ/3. ТЕРМИНЫ И СОКРАЩЕНИЯ', 'bu..."
1,"Уважаемые коллеги, Обращаюсь с вопросом: При ...",1. С кем необходимо согласовать диапазон цен п...,Согласно СК-03.03.02.01 Проведение закупки у е...,"Для ответа на первый вопрос, необходимо обрати...",[{'source': 'ПОЗ/5. НАЗНАЧЕНИЕ И СТРУКТУРА ПРО...
2,"Уважаемые коллеги, У меня вопрос, связанный с...",1. Может ли быть заключен договор с единственн...,"пункт 17.1.8. в случае, когда на участие в кон...","Да, договор может быть заключен с единственным...",[{'source': 'ПОЛОЖЕНИЕ/7. ПОРЯДОК ПОДГОТОВКИ И...


In [583]:
responses.to_excel('rag_responses.xlsx', index=False)

## Basic Queries | Neo4j

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [ ]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

In [ ]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [ ]:
cypher = """
  SHOW VECTOR INDEXES
  """

graph.query(cypher)

[{'id': 5,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 16, 13, 21, 38, 725000000, tzinfo=<UTC>),
  'readCount': 43}]

In [ ]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """

graph.query(cypher)

[{'count(n)': 1030}]

In [ ]:
cypher = """
    MATCH (n:Chunk {bullet: "1.3.1"})
    RETURN n.text AS text
    """

graph.query(cypher)

[{'text': '1.3.1. Планирование закупок ПАО «Компания 1» и Обществ Компания 1, в рамках  которого:'}]